---
title: "odc.stack.stac_load"
categories: [data-download]
date: 2025-04-24
---

## Query Sentinel-2 data via odc.stac.stac_load into xarray.Dataset

In [ ]:
from pystac_client import Client
import planetary_computer
from odc.stac import stac_load

from bounding_box import *

*Set `modifier` in client, otherwise the search will succeed, but data access not*

In [ ]:
catalog = Client.open(
    "https://planetarycomputer.microsoft.com/api/stac/v1",
    modifier=planetary_computer.sign_inplace,
)

query = catalog.search(
    collections=["sentinel-2-l2a"],
    bbox=[lon_min, lat_min, lon_max, lat_max],
    datetime="2024-07-01",
)

items = list(query.items())
print(f"Found: {len(items):d} datasets")

*Remember to set the `chunks`, otherwise the data will be loaded eagerly!*

In [ ]:
chunk_size = 1024

ds = stac_load(
    items,
    bands=("B04", "B03", "B02", "B08"),
    resolution=10,
    chunks={"time": 1, "x": chunk_size, "y": chunk_size} # this prevents eager loading of data
)
ds

In [ ]:
ds = ds.sel(x=slice(x_min, x_max), y=slice(y_max, y_min))

In [ ]:
ds = ds.isel(time=[0]) # just take first timestep to reduce amount of data

In [ ]:
# load data into memory if desired
ds.load()

In [ ]:
ds["B04"].squeeze().plot.imshow()

... write to disk, if necessary ...